In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

#### 정규화, 1열 만들기(CNN에서는 안 함 - 현업에서는 어짜피 CNN쓰니까 할 일 없음)

In [4]:
train_scaled = train_input / 255.0
test_scaled = test_input / 255.0

# reshape
train_scaled = train_scaled.reshape(-1, 28*28)
test_scaled = test_scaled.reshape(-1, 28*28)

In [5]:
train_scaled.shape

(60000, 784)

In [6]:
# train과 valid 분리
from sklearn.model_selection import train_test_split
train_scaled, valid_scaled, train_target, valid_target = train_test_split(
  train_scaled, train_target, test_size=0.2, random_state=42)

## Layer를 추가하는 방법-1

In [7]:
# Layer부터 만들기
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) 
#은닉층 activation함수 - sigmoid 써도 됨 (출력층은 다중분류라 sigmoid 못써)

dense2 = keras.layers.Dense(10, activation='softmax')
# 다중분류라 소프트맥스 써야 함

In [8]:
# 전체 모델에 층을 추가하며 만들기
model = keras.models.Sequential([dense1, dense2])

2022-07-20 15:13:37.078712: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# 레이어 잘 구성 되었는지 Summary
model.summary()

# 784개가 들어왔는데 78400이 아니라 78500인 이유: b가 더해져서 무조건 1씩 늘어나

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


##### 784개 -> (784+1) * 100 -> 78500
##### 100개 -> (100+1) * 10 -> 1010

## Layer를 추가하는 방법-2

In [10]:
# 한 번에 모델과 레이어 다 만들기
model = keras.models.Sequential([
  keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
  keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 Mnist 모델')

In [11]:
model.summary() # 이름 준 것들이 보임

Model: "패션 Mnist 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
## Layer를 추가하는 방법 - 3 (주로 씀)

#### 주로 쓰는 이유: 층이 데이터에 따라 바뀜 > if문을 써야 하는데 2번에서는 if를 쓸 방법이 없음

In [12]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
# 모델 훈련

In [13]:
model.compile(loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5625 - accuracy: 0.8068
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4065 - accuracy: 0.8533
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3707 - accuracy: 0.8672
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3480 - accuracy: 0.8744
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3305 - accuracy: 0.8805


### 시그모이드와 렐루
- 0~1까지 구분되는건 좋은데 x가 커지면 1로 너무 많이 붙어서(0.99999999, 0.999999999....) 구분이 잘 안 됨
- relu? 함수는 그런게 없어서 x가 커져도 구분 잘 됨 > 해결! 뾰롱 > 인공신경망에 기여한 바가 크다고 함

In [15]:
# 입력층(784개)을 단순화
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28))) # 입력층을 2차원 원래대로 사용한다. > 알아서 펴줌
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
## Optimizer(최적화)
- 훈련 시 Optimizer를 사용하여 최적화 알고리즘을 구성한다.
- 보통 Optimizer로 사용하는 것이 확률적 경사하강법이다.

In [ ]:
# 적용방법(확률적 경사하강법) - 1
model.compile(loss='sparse_categorical_crossentropy',
              metrics=['accuracy'], optimizer='sgd')

##### SGD의 기본값은 lr(learning rate)=0.01

In [19]:
# 적용방법(확률적 경사하강법) - 2
sgd = keras.optimizers.SGD(lr=0.1)
model.compile(loss='sparse_categorical_crossentropy',
              metrics=['accuracy'], optimizer=sgd)

In [17]:
# 적용방법(확률적 경사하강법) - 3 - Momentum 적용
sgd2 = keras.optimizers.SGD(momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy',
              metrics=['accuracy'], optimizer=sgd)

##### 모멘텀: 빠른 속도로 일단 다 하강해보고 다시 올라오면서 최솟값 찾음

In [30]:
# 적용방법(Adagrad)
model.compile(loss='sparse_categorical_crossentropy',
              metrics=['accuracy'], optimizer='adagrad')

In [28]:
# 적용방법(RMSprop) 
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='rmsprop')

In [29]:
# 적용방법(Adadelta)
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='adadelta')

In [32]:
# 적용방법(Adam)
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [26]:
# 적용방법(Nadam)
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='nadam')